In [1]:
## DONT CHANGE THIS CELL 
# this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

--2021-10-25 17:31:15--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt [following]
--2021-10-25 17:31:16--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63639 (62K) [text/plain]
Saving to: ‘test.data.txt’

test.data.txt       100%[===================>]  62.15K  86.0KB/s    in 0.7s    

2021-10-25 17:31:18 (86.0 KB/s) - ‘test.data.txt’ saved [63639/63639]



In [2]:
!gdown https://drive.google.com/uc?id=11BItpIV77eMRIAEIbvzb9t4F-H--IHuj

Downloading...
From: https://drive.google.com/uc?id=11BItpIV77eMRIAEIbvzb9t4F-H--IHuj
To: /content/2018EE10493_B_model.zip
100% 1.48G/1.48G [00:14<00:00, 104MB/s] 


In [3]:
!unzip ./2018EE10493_B_model.zip

Archive:  ./2018EE10493_B_model.zip
 extracting: data.zip                
   creating: FINAL_MODEL/
   creating: FINAL_MODEL/model/
  inflating: FINAL_MODEL/model/config.json  
  inflating: FINAL_MODEL/model/pytorch_model.bin  
  inflating: FINAL_MODEL/model/training_args.bin  
   creating: FINAL_MODEL/tokenizer/
  inflating: FINAL_MODEL/tokenizer/tokenizer_config.json  
  inflating: FINAL_MODEL/tokenizer/special_tokens_map.json  
  inflating: FINAL_MODEL/tokenizer/vocab.txt  
  inflating: FINAL_MODEL/tokenizer/tokenizer.json  
   creating: NLP_A3/
   creating: NLP_A3/data/
   creating: NLP_A3/data/train/
  inflating: NLP_A3/data/train/train.gold.txt  
  inflating: NLP_A3/data/train/train.data.txt  
   creating: NLP_A3/data/validation/
  inflating: NLP_A3/data/validation/validation.data.txt  
  inflating: NLP_A3/data/validation/validation.gold.txt  
   creating: saved_model/
   creating: saved_model/runs/
   creating: saved_model/runs/Oct25_16-56-16_e6d9d5d3a244/
  inflating: saved_mod

In [4]:
test_data_path = '/content/test.data.txt'

In [5]:
## Import relevant packages

import os

import time
import datetime

import torch
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

import logging
from argparse import ArgumentParser

from pdb import set_trace

In [6]:
import zipfile
from torchtext.legacy.data import Field, BucketIterator, TabularDataset
import spacy
import pandas as pd
import numpy as np
import random

In [7]:
! pip install transformers

     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 42.4 MB/s 
     |████████████████████████████████| 3.3 MB 34.1 MB/s 
     |████████████████████████████████| 56 kB 3.6 MB/s 
     |████████████████████████████████| 596 kB 56.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [9]:
tokenizer_test = AutoTokenizer.from_pretrained("/content/FINAL_MODEL/tokenizer")

In [10]:
model_test = AutoModelForSequenceClassification.from_pretrained("/content/FINAL_MODEL/model",num_labels=2 )

In [11]:
import pandas as pd

In [12]:
from torch.utils.data import Dataset

In [13]:
class Customed_Dataset(Dataset):
  def __init__(self, X, y = None):
    self.text_encodings = X
    self.labels = y
  
  def __getitem__(self, index):
    sample = {k: torch.tensor(v[index]) for k , v in self.text_encodings.items()}
    if self.labels:
      sample['labels'] = torch.tensor(self.labels[index])

    return sample
  
  def __len__(self):
    return len(self.text_encodings['input_ids'])

In [14]:
def Dataset(tokenizer):
  file_v = pd.read_csv(test_data_path,sep='\t', header = None, names = ['word', 'POS', 'index_sent1', 'sent1', 'sent2'])
  X_test = tokenizer(list(file_v['sent1']), list(file_v['sent2']) , padding = True, truncation = True)
  
  test_dataset = Customed_Dataset(X_test)

  return test_dataset

In [15]:
test_dataset = Dataset(tokenizer_test)

In [16]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [17]:
def compute_metrics(eval_pred):
    pred, labels = eval_pred
    pred = np.argmax(pred, axis=1)
    dict_metric = {}

    dict_metric["accuracy"] = accuracy_score(y_true=labels, y_pred=pred)
    dict_metric["recall"] = recall_score(y_true=labels, y_pred=pred)
    dict_metric["precision"] = precision_score(y_true=labels, y_pred=pred)
    dict_metric["f1"] = f1_score(y_true=labels, y_pred=pred)

    return dict_metric

In [18]:
trainer1 = Trainer(model = model_test,compute_metrics=compute_metrics)

In [19]:
output, _, _ = trainer1.predict(test_dataset)
y_pred = np.argmax(output, axis = 1)

***** Running Prediction *****
  Num examples = 638
  Batch size = 8


In [20]:
with open('output.txt', "w") as output_file:
  for y in y_pred:
    if y == 1:
      Y_write = 'T'
    elif y == 0:
      Y_write = 'F'
    output_file.write("%s\n" % Y_write)
  output_file.close()

In [21]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
  prediction, gold = prediction.strip(), gold.strip()
  total += 1
  if prediction == gold:
    correct += 1

## Report this as the final validation performance 
print('Performance = ', (correct/total))

--2021-10-25 17:35:34--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt [following]
--2021-10-25 17:35:35--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276 (1.2K) [text/plain]
Saving to: ‘test.gold.txt’

test.gold.txt       100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-10-25 17:35:36 (34.8 MB/s) - ‘test.gold.txt’ saved [1276/1276]

Performance =  0.7053291536050157
